In [1]:
import cv2,os

data_path='dataset'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'with mask': 0, 'without mask': 1}
['with mask', 'without mask']
[0, 1]


In [2]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            #Coverting the image into gray scale
            resized=cv2.resize(gray,(img_size,img_size))
            #resizing the gray scale into 50x50, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)

        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

In [3]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)



np.save('data',data)
np.save('target',new_target)



Using TensorFlow backend.


In [4]:
target

array([0, 0, 0, ..., 1, 1, 1])

In [5]:
new_target

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [6]:
import numpy as np

data=np.load('data.npy')
target=np.load('target.npy')

In [7]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

In [8]:
model=Sequential()

In [9]:
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [10]:
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [11]:
model.add(Flatten())
model.add(Dropout(0.5))

In [12]:
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [14]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Train on 990 samples, validate on 248 samples
Epoch 1/20
990/990 [==============================] - 52s 53ms/step - loss: 0.7774 - accuracy: 0.5323 - val_loss: 0.6439 - val_accuracy: 0.7016
Epoch 2/20
990/990 [==============================] - 50s 51ms/step - loss: 0.5664 - accuracy: 0.7212 - val_loss: 0.4452 - val_accuracy: 0.8427
Epoch 3/20
990/990 [==============================] - 53s 54ms/step - loss: 0.4494 - accuracy: 0.7828 - val_loss: 0.4652 - val_accuracy: 0.7581
Epoch 4/20
990/990 [==============================] - 54s 54ms/step - loss: 0.3610 - accuracy: 0.8414 - val_loss: 0.3131 - val_accuracy: 0.8831
Epoch 5/20
990/990 [==============================] - 52s 53ms/step - loss: 0.2785 - accuracy: 0.8909 - val_loss: 0.2465 - val_accuracy: 0.8992
Epoch 6/20
990/990 [==============================] - 53s 54ms/step - loss: 0.2181 - accuracy: 0.9182 - val_loss: 0.2121 - val_accuracy: 0.9315
Epoch 7/20
990/990 [==============================] - 53s 54ms/step - loss: 0.1692 - accur